In [2]:
# Crear las opciones
options = ["piedra", "tijeras", "papel"]

In [3]:
def search_winner(p1, p2):
    if p1 == p2:
        result = 0
    
    elif p1 == "piedra" and p2 == "tijeras":
        result = 1
    elif p1 == "piedra" and p2 == "papel":
        result = 2
    elif p1 == "tijeras" and p2 == "piedra":
        result = 2
    elif p1 == "tijeras" and p2 == "papel":
        result = 1
    elif p1 == "papel" and p2 == "piedra":
        result = 1
    elif p1 == "papel" and p2 == "tijeras":
        result = 2
        
    return result

In [4]:
search_winner("papel", "tijeras")

2

In [5]:
test = [
    ["piedra", "piedra", 0],
    ["piedra", "tijeras", 1],
    ["piedra", "papel", 2]
]

for partida in test:
    print("player1: %s player2: %s Winner: %s Validation: %s" % (
        partida[0], partida[1], search_winner(partida[0], partida[1]), partida[2]
    ))

player1: piedra player2: piedra Winner: 0 Validation: 0
player1: piedra player2: tijeras Winner: 1 Validation: 1
player1: piedra player2: papel Winner: 2 Validation: 2


In [6]:
from random import choice
def get_choice():
    return choice(options)

In [44]:
get_choice()

'papel'

In [8]:
for i in range(10):
    # player1 = get_choice()
    player2 = get_choice()
    print("player1: %s player2: %s Winner: %s " % (
        player1, player2, search_winner(player1, player2)
    ))

player1: tijeras player2: tijeras Winner: 0 
player1: papel player2: papel Winner: 0 
player1: piedra player2: piedra Winner: 0 
player1: papel player2: piedra Winner: 1 
player1: papel player2: papel Winner: 0 
player1: piedra player2: tijeras Winner: 1 
player1: papel player2: piedra Winner: 1 
player1: piedra player2: tijeras Winner: 1 
player1: piedra player2: papel Winner: 2 
player1: piedra player2: tijeras Winner: 1 


In [9]:
def str_to_list(option):
    if option=="piedra":
        res = [1,0,0]
    elif option=="tijeras":
        res = [0,1,0]
    else:
        res = [0,0,1]
    return res

data_X = list(map(str_to_list, ["piedra", "tijeras", "papel"]))
data_y = list(map(str_to_list, ["papel", "piedra", "tijeras"]))

print(data_X)
print(data_y)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[0, 0, 1], [1, 0, 0], [0, 1, 0]]


In [10]:
from sklearn.neural_network import MLPClassifier

In [11]:
clf = MLPClassifier(verbose=False, warm_start=True)

In [12]:
model = clf.fit([data_X[0]], [data_y[0]])
print(model)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=True)


C:\Users\rguerrerop\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [59]:
def play_and_learn(iters=10, debug=False):
    score = {"win": 0, "loose": 0}
    
    data_X = []
    data_y = []
    
    for i in range(iters):
        player1 = 'papel'
        
        predict = model.predict_proba([str_to_list(player1)])[0]
        
        if predict[0] >= 0.95:
            player2 = options[0]
        elif predict[1] >= 0.95:
            player2 = options[1]
        elif predict[2] >= 0.95:
            player2 = options[2]
        else:
            player2 = get_choice()
            
        if debug==True:
            print("Player1: %s Player2 (modelo): %s --> %s" % (player1, predict, player2))
        
        winner = search_winner(player1, player2)
        if debug==True:
            print("Comprobamos: p1 VS p2: %s" % winner)
        
        if winner==2:
            data_X.append(str_to_list(player1))
            data_y.append(str_to_list(player2))
            
            score["win"]+=1
        else:
            score["loose"]+=1
        
    return score, data_X, data_y

In [60]:
score, data_X, data_y = play_and_learn(1, debug=True)
print(data_X)
print(data_y)
print("Score: %s %s %%" % (score, (score["win"]*100/(score["win"]+score["loose"]))))
if len(data_X):
    model = model.partial_fit(data_X, data_y)

Player1: papel Player2 (modelo): [0.01249148 0.98848234 0.0069314 ] --> tijeras
Comprobamos: p1 VS p2: 2
[[0, 0, 1]]
[[0, 1, 0]]
Score: {'win': 1, 'loose': 0} 100.0 %


In [39]:
i = 0
historic_pct = []
while True:
    i+=1
    score, data_X, data_y = play_and_learn(1000, debug=False)
    pct = (score["win"]*100/(score["win"]+score["loose"]))
    historic_pct.append(pct)
    print("Iter: %s - score: %s %s %%" % (i, score, pct))
    
    if len(data_X):
        model = model.partial_fit(data_X, data_y)
    
    if sum(historic_pct[-9:])==900:
        break

Iter: 1 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 2 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 3 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 4 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 5 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 6 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 7 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 8 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 9 - score: {'win': 1000, 'loose': 0} 100.0 %


In [16]:
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, show, output_notebook
output_notebook()

Loading BokehJS ...

In [17]:
x = range(len(historic_pct))
y = historic_pct

p = figure(
    title="Porcetaje de aprendizaje en cada iteración",
    x_axis_label="Iter", y_axis_label="%", width=900)

p.line(x, y, legend=None, line_width=1)
show(p)